In [ ]:
!pip install -q transformers datasets evaluate sentence-transformers scikit-learn pandas matplotlib seaborn imbalanced-learn lime shap gradio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.2 MB/s eta 0:00:00


In [ ]:
import kagglehub


path = kagglehub.dataset_download("infamouscoder/depression-reddit-cleaned")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'depression-reddit-cleaned' dataset.
Path to dataset files: /kaggle/input/depression-reddit-cleaned


In [ ]:
from google.colab import files
import pandas as pd


uploaded = files.upload()


df = pd.read_csv(list(uploaded.keys())[0])

print(df.columns)


print(df['is_depression'].value_counts())


Saving depression_dataset_reddit_cleaned.csv to depression_dataset_reddit_cleaned (4).csv
Index(['clean_text', 'is_depression'], dtype='object')
is_depression
0    3900
1    3831
Name: count, dtype: int64


In [ ]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"@\w+", "", text)
    text = re.sub(r"#\w+", "", text)
    text = re.sub(r"[^a-z\s]", "", text)
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(w) for w in tokens if w not in stop_words]
    return " ".join(tokens)


df['clean_text'] = df['clean_text'].apply(clean_text)
df.head()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


,clean_text,is_depression
0,understand people reply immediately op invitat...,1
1,welcome r depression check post place take mom...,1
2,anyone else instead sleeping depressed stay ni...,1
3,kind stuffed around lot life delaying inevitab...,1
4,sleep greatest comforting escape whenever wake...,1


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report


X_train, X_test, y_train, y_test = train_test_split(
    df['clean_text'], df['is_depression'], test_size=0.2, random_state=42
)


vectorizer = TfidfVectorizer(max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)


model = LogisticRegression()
model.fit(X_train_vec, y_train)


y_pred = model.predict(X_test_vec)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.93      0.98      0.95       783
           1       0.98      0.92      0.95       764

    accuracy                           0.95      1547
   macro avg       0.95      0.95      0.95      1547
weighted avg       0.95      0.95      0.95      1547



In [ ]:
sample = "I feel sad and hopeless all the time."
sample_vec = vectorizer.transform([clean_text(sample)])
print("Prediction:", model.predict(sample_vec)[0])


Prediction: 0


In [ ]:
import joblib

joblib.dump(model, "mental_health_model.pkl")
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")


['tfidf_vectorizer.pkl']

In [ ]:

sample = "I feel sad and hopeless all the time."


sample_cleaned = clean_text(sample)


sample_vec = vectorizer.transform([sample_cleaned])

l.predict(sample_vec)[0]


label_map = {0: "Not Depressed", 1: "Depressed"}
print("Prediction:", label_map[pred])


Prediction: Not Depressed


In [ ]:
def predict_mental_health(text):
    vec = vectorizer.transform([clean_text(text)])
    pred = model.predict(vec)[0]
    return label_map[pred]

print(predict_mental_health("I feel sad and hopeless all the time."))


Not Depressed
